In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# from s
# from sklearn.
# from sklearn.decomposition import PCA 

In [2]:
data=pd.read_csv(r"C:\Users\vvagh\OneDrive - Indian Institute of Science Education and Research Bhopal\Documents\IITM Stuff\diploma-LittleBeasty\MLP project\T124OPPE2_Preprocessing_V1.csv")
data

,Gender,Age,HasTension,AnyHeartDisease,NeverMarried,Occupation,LivesIn,GlucoseLevel,BMI,SmokingStatus,HeartAttack
0,Female,75.0,Yes,No,Yes,Self-employed,City,54.6,35.1,never smoked,No
1,Female,49.0,No,No,Yes,Private,Village,108.8,26.7,smokes,No
2,Male,32.0,No,No,Yes,Private,City,64.1,23.4,smokes,No
3,Male,78.0,No,No,Yes,Self-employed,City,219.2,27.4,Unknown,Yes
4,Male,39.0,No,No,Yes,Private,City,55.4,41.6,formerly smoked,No
...,...,...,...,...,...,...,...,...,...,...,...
3995,Female,40.0,No,No,Yes,Private,City,88.4,36.5,smokes,No
3996,Female,18.0,No,No,No,Private,Village,168.5,48.2,never smoked,No
3997,Male,27.0,No,No,Yes,Private,City,76.5,21.0,never smoked,No
3998,Female,28.0,No,No,No,Private,City,80.0,27.1,smokes,No


In [3]:
data.isna().sum()

Gender               0
Age                  0
HasTension           0
AnyHeartDisease      0
NeverMarried         0
Occupation           0
LivesIn              0
GlucoseLevel         0
BMI                149
SmokingStatus        0
HeartAttack          0
dtype: int64

In [4]:
data['SmokingStatus'].unique()

array(['never smoked', 'smokes', 'Unknown', 'formerly smoked'],
      dtype=object)

In [5]:
len(data[data['SmokingStatus']=='Unknown'].index)

1204

In [6]:
data['BMI'].mean()

np.float64(28.857958971695663)

In [7]:
data[data['BMI'].isna()]

,Gender,Age,HasTension,AnyHeartDisease,NeverMarried,Occupation,LivesIn,GlucoseLevel,BMI,SmokingStatus,HeartAttack
15,Female,20.0,No,No,No,Govt_job,Village,80.4,NaN,never smoked,No
61,Male,58.0,No,No,Yes,Private,City,93.1,NaN,Unknown,No
93,Female,80.0,Yes,No,Yes,Private,City,84.2,NaN,never smoked,No
100,Female,61.0,No,No,Yes,Private,Village,219.1,NaN,never smoked,No
101,Female,75.0,No,No,Yes,Govt_job,City,59.7,NaN,Unknown,Yes
...,...,...,...,...,...,...,...,...,...,...,...
3823,Female,67.0,Yes,No,Yes,Private,Village,83.7,NaN,smokes,No
3826,Female,69.0,Yes,No,Yes,Govt_job,City,111.7,NaN,never smoked,No
3856,Male,61.0,No,No,Yes,Govt_job,City,184.9,NaN,Unknown,No
3863,Male,58.0,No,No,Yes,Private,City,70.5,NaN,Unknown,Yes


In [8]:
data.apply(lambda x: (x == 'Unknown').sum())

Gender                7
Age                   0
HasTension            0
AnyHeartDisease       0
NeverMarried          0
Occupation            0
LivesIn               5
GlucoseLevel          0
BMI                   0
SmokingStatus      1204
HeartAttack           0
dtype: int64

In [9]:
pipe=Pipeline([('imputer',SimpleImputer(strategy='mean')),('scaler',StandardScaler())])
pipe_cat=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encode',OneHotEncoder())
])
couln_transform = ColumnTransformer([
    ('num', pipe, [col for col in data.columns if data[col].dtype in ['int64', 'float64']]),
    ('cat', pipe_cat, [col for col in data.columns if data[col].dtype == 'object'])
])

In [10]:
# Fit the ColumnTransformer
couln_transform.fit(data)

# Get the feature names
feature_names = couln_transform.get_feature_names_out()

# Strip off the prefixes 'num__' and 'cat__'
clean_feature_names = [name.split('__')[1] if '__' in name else name for name in feature_names]

# Create a new DataFrame with the cleaned feature names
new_Data = pd.DataFrame(couln_transform.transform(data), columns=clean_feature_names)

# remove the redundent columns, i.e only one of htem is required for a binary category varibale.

In [11]:
new_Data

,Age,GlucoseLevel,BMI,Gender_Female,Gender_Male,Gender_Unknown,HasTension_No,HasTension_Yes,AnyHeartDisease_No,AnyHeartDisease_Yes,...,Occupation_children,LivesIn_City,LivesIn_Unknown,LivesIn_Village,SmokingStatus_Unknown,SmokingStatus_formerly smoked,SmokingStatus_never smoked,SmokingStatus_smokes,HeartAttack_No,HeartAttack_Yes
0,1.402582,-1.130212,0.804933,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.257300,0.061223,-0.278276,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-0.491538,-0.921381,-0.703823,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.534730,2.488057,-0.188009,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,-0.183193,-1.112626,1.643130,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-0.139144,-0.387214,0.985467,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3996,-1.108229,1.373560,2.494223,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3997,-0.711785,-0.648802,-1.013311,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3998,-0.667736,-0.571864,-0.226695,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [12]:
data

,Gender,Age,HasTension,AnyHeartDisease,NeverMarried,Occupation,LivesIn,GlucoseLevel,BMI,SmokingStatus,HeartAttack
0,Female,75.0,Yes,No,Yes,Self-employed,City,54.6,35.1,never smoked,No
1,Female,49.0,No,No,Yes,Private,Village,108.8,26.7,smokes,No
2,Male,32.0,No,No,Yes,Private,City,64.1,23.4,smokes,No
3,Male,78.0,No,No,Yes,Self-employed,City,219.2,27.4,Unknown,Yes
4,Male,39.0,No,No,Yes,Private,City,55.4,41.6,formerly smoked,No
...,...,...,...,...,...,...,...,...,...,...,...
3995,Female,40.0,No,No,Yes,Private,City,88.4,36.5,smokes,No
3996,Female,18.0,No,No,No,Private,Village,168.5,48.2,never smoked,No
3997,Male,27.0,No,No,Yes,Private,City,76.5,21.0,never smoked,No
3998,Female,28.0,No,No,No,Private,City,80.0,27.1,smokes,No


In [13]:
group = data.groupby(['LivesIn', 'SmokingStatus', 'HeartAttack']).apply(
    lambda x: x[(x['LivesIn'] == 'City') & 
                ((x['SmokingStatus'] == 'formerly smoked') | (x['SmokingStatus'] == 'smokes')) & 
                (x['HeartAttack'] == 'Yes')])

group.idxmax()

C:\Users\vvagh\AppData\Local\Temp\ipykernel_24472\2827419205.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group = data.groupby(['LivesIn', 'SmokingStatus', 'HeartAttack']).apply(


Gender             (City, formerly smoked, Yes, 386)
Age                (City, formerly smoked, Yes, 274)
HasTension         (City, formerly smoked, Yes, 274)
AnyHeartDisease    (City, formerly smoked, Yes, 274)
NeverMarried       (City, formerly smoked, Yes, 274)
Occupation         (City, formerly smoked, Yes, 929)
LivesIn            (City, formerly smoked, Yes, 274)
GlucoseLevel               (City, smokes, Yes, 3928)
BMI                (City, formerly smoked, Yes, 632)
SmokingStatus               (City, smokes, Yes, 288)
HeartAttack        (City, formerly smoked, Yes, 274)
dtype: object

In [14]:
clean_data = data
filtered_data = clean_data[
    ((clean_data['LivesIn'] == 'City') ) &
    ((clean_data['SmokingStatus'] == 'formerly smoked') | (clean_data['SmokingStatus'] == 'smokes')) &
    (clean_data['HeartAttack'] == 'Yes')
]
filtered_data.shape

(52, 11)

In [15]:
# Which of the following categories have highest frequency? Ignore rows with missing values
freq=[i for i in data.columns if data[i].dtype=='object' and data[i].value_counts().idxmax()]


In [16]:
freq

['Gender',
 'HasTension',
 'AnyHeartDisease',
 'NeverMarried',
 'Occupation',
 'LivesIn',
 'SmokingStatus',
 'HeartAttack']

In [17]:
female_freq=data[data['Gender']=='Female']['SmokingStatus'].value_counts().idxmax()

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           4000 non-null   object 
 1   Age              4000 non-null   float64
 2   HasTension       4000 non-null   object 
 3   AnyHeartDisease  4000 non-null   object 
 4   NeverMarried     4000 non-null   object 
 5   Occupation       4000 non-null   object 
 6   LivesIn          4000 non-null   object 
 7   GlucoseLevel     4000 non-null   float64
 8   BMI              3851 non-null   float64
 9   SmokingStatus    4000 non-null   object 
 10  HeartAttack      4000 non-null   object 
dtypes: float64(3), object(8)
memory usage: 343.9+ KB


In [19]:
distribution = data['HeartAttack'].value_counts()
print(distribution)

HeartAttack
No     3806
Yes     194
Name: count, dtype: int64


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming data is already loaded in 'data'

# Step 1: Define features and target
X = data.drop(columns=['HeartAttack'])
y = data['HeartAttack'].replace({'Yes': 1, 'No': 0})  # Replace "Yes" with 1 and "No" with 0

# Step 2: Split the data into training and test sets (30% test set, stratified on target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Step 3: Preprocessing pipeline
# Define the preprocessing for each column
preprocessor = ColumnTransformer(
    transformers=[
        ('gender', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder())
        ]), ['Gender']),
        
        ('age', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), ['Age']),
        
        ('has_tension', Pipeline([
            ('encoder', OrdinalEncoder())
        ]), ['HasTension']),
        
        ('any_heart_disease', Pipeline([
            ('encoder', OrdinalEncoder())
        ]), ['AnyHeartDisease']),
        
        ('never_married', Pipeline([
            ('encoder', OrdinalEncoder())
        ]), ['NeverMarried']),
        
        ('occupation', Pipeline([
            ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
        ]), ['Occupation']),
        
        ('lives_in', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder())
        ]), ['LivesIn']),
        
        ('glucose_level', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', MinMaxScaler())
        ]), ['GlucoseLevel']),
        
        ('bmi', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), ['BMI']),
        
        ('smoking_status', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('one_hot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
        ]), ['SmokingStatus'])
    ]
)

# Step 4: Apply the transformations using the pipeline
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Step 5: Get the shape of the feature matrix for the training dataset
print(f"Shape of the training feature matrix: {X_train_processed.shape}")


C:\Users\vvagh\AppData\Local\Temp\ipykernel_24472\3443754012.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = data['HeartAttack'].replace({'Yes': 1, 'No': 0})  # Replace "Yes" with 1 and "No" with 0


Shape of the training feature matrix: (2800, 17)


In [21]:
# model building
df=pd.read_csv(r"C:\Users\vvagh\OneDrive - Indian Institute of Science Education and Research Bhopal\Documents\IITM Stuff\diploma-LittleBeasty\MLP project\T124OPPE2_ModelBuilding_V1.csv")
df

,Gender,Age,HasTension,AnyHeartDisease,NeverMarried,Occupation_Govt_job,Occupation_Never_worked,Occupation_Private,Occupation_Self-employed,Occupation_children,LivesIn,GlucoseLevel,BMI,SmokingStatus_formerly smoked,SmokingStatus_never smoked,SmokingStatus_smokes,HeartAttack
0,0.0,0.433901,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.203263,0.284421,0.0,1.0,0.0,0
1,1.0,-1.840435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.386014,-1.186687,0.0,1.0,0.0,0
2,1.0,-1.160260,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.326807,0.065045,0.0,1.0,0.0,0
3,1.0,-0.806002,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.182284,0.929643,0.0,1.0,0.0,0
4,0.0,0.743876,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.200466,-0.412420,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.0,0.389618,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.099767,0.774790,1.0,0.0,0.0,0
3996,1.0,1.452392,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.657809,-0.373706,1.0,0.0,0.0,1
3997,0.0,0.433901,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.041958,0.271516,0.0,1.0,0.0,1
3998,1.0,0.921005,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.140793,0.155376,0.0,1.0,0.0,0


In [22]:
new_Data

,Age,GlucoseLevel,BMI,Gender_Female,Gender_Male,Gender_Unknown,HasTension_No,HasTension_Yes,AnyHeartDisease_No,AnyHeartDisease_Yes,...,Occupation_children,LivesIn_City,LivesIn_Unknown,LivesIn_Village,SmokingStatus_Unknown,SmokingStatus_formerly smoked,SmokingStatus_never smoked,SmokingStatus_smokes,HeartAttack_No,HeartAttack_Yes
0,1.402582,-1.130212,0.804933,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.257300,0.061223,-0.278276,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-0.491538,-0.921381,-0.703823,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.534730,2.488057,-0.188009,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,-0.183193,-1.112626,1.643130,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-0.139144,-0.387214,0.985467,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3996,-1.108229,1.373560,2.494223,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3997,-0.711785,-0.648802,-1.013311,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3998,-0.667736,-0.571864,-0.226695,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [23]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split

# Assuming data is already loaded in 'data' and target is 'HeartAttack'
X = df.drop(columns=['HeartAttack'])
y = df['HeartAttack'].replace({'Yes': 1, 'No': 0})  # Replace "Yes" with 1 and "No" with 0

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=False)

# Initialize the Perceptron classifier with the specified parameters
perceptron = Perceptron(
    random_state=1729,
    eta0=1,  # learning rate
    max_iter=5,  # number of iterations (epochs)
    shuffle=False,  # do not shuffle dataset
    fit_intercept=True,  # include intercept term
    validation_fraction=0.1,  # 10% of the data for validation
    penalty=None,  # no regularization
    warm_start=True  # keep the learned coefficients for the next fit
)

# Train the Perceptron classifier
perceptron.fit(X_train, y_train)

# Get the value of the bias (intercept) after 5th iteration
bias_after_5th_iteration = perceptron.intercept_
print(f"Bias (Intercept) after 5th iteration: {bias_after_5th_iteration}")

Bias (Intercept) after 5th iteration: [-3.]


c:\Users\vvagh\miniconda3\envs\stenv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [24]:
perceptron.get_params()

{'alpha': 0.0001,
 'class_weight': None,
 'early_stopping': False,
 'eta0': 1,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'max_iter': 5,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': None,
 'random_state': 1729,
 'shuffle': False,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': True}

In [25]:
perceptron.coef_

array([[ 0.        ,  0.24850288, -1.        ,  3.        ,  0.        ,
        -1.        ,  0.        , -1.        , -2.        ,  1.        ,
        -1.        ,  1.58181818, -1.09161751, -2.        , -1.        ,
         0.        ]])

In [26]:
perceptron.intercept_

array([-3.])

In [27]:
from sklearn.metrics import precision_score

# Check the distribution of target classes in the training set
print("Training data class distribution:")
print(y_train.value_counts())

# Check the first few predictions
y_train_pred = perceptron.predict(X_train)
print("First 20 predictions:", y_train_pred[:20])

# Compute the precision for the positive class (class value 1)
precision = precision_score(y_train, y_train_pred, pos_label=1)

print(f"Precision for positive class after 5 iterations: {precision:.2f}")

Training data class distribution:
HeartAttack
0    2664
1     136
Name: count, dtype: int64
First 20 predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Precision for positive class after 5 iterations: 0.67


In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier


# Initialize SGDClassifier with the given parameters
sgd = SGDClassifier(loss='log_loss', learning_rate='constant', random_state=1729, eta0=0.001, max_iter=1, alpha=0, penalty='l2')
# sgd.
# saving the scores for 5 iterations 
scores = []

# Train the model for 5 iterations
for _ in range(5):
    sgd.partial_fit(X_train, y_train, classes=np.unique(y))
    scores.append(sgd.score(X_train, y_train))
scores
# Define the parameter grid
# param_grid = {
    # 'alpha': [0.0001, 0.0005, 0.001, 0.005],
    # 'eta0': [0.01, 0.05, 0.1, 0.5]
# }

# Set up GridSearchCV
# grid_search = GridSearchCV(estimator=sgd, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)  # Replace X_train and y_train with your dataset

# Get the best parameters
# best_params = grid_search.best_params_
# print(best_params)


[0.9514285714285714,
 0.9514285714285714,
 0.9514285714285714,
 0.9514285714285714,
 0.9514285714285714]

In [29]:
# solution for Q4
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

# Generate a simple dataset
X, y = make_classification(n_samples=1000, n_features=20, random_state=1729)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1729)

# Initialize the SGDClassifier
clf = SGDClassifier(loss='log_loss', penalty='l2', eta0=0.001, alpha=0, learning_rate='constant',
                    random_state=1729, warm_start=True, max_iter=1, tol=None)

# Training loop for 5 iterations
losses = []
for i in range(5):
    clf.fit(X_train, y_train)
    y_pred = clf.decision_function(X_train)
    loss = np.mean(np.log(1 + np.exp(-y_train * y_pred)))  # Log-loss computation
    losses.append(loss)

# Loss after second iteration
print(f"Loss after second iteration: {losses[1]:.3f}")


Loss after second iteration: 0.529


In [30]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix

# Use the best parameters from GridSearchCV (replace with the actual best values found)
best_alpha = 0.001  # Replace with the best alpha found
best_eta0 = 0.1     # Replace with the best eta0 found

# Initialize SGDClassifier with the best parameters
sgd_best = SGDClassifier(
    loss='log_loss', 
    learning_rate='constant', 
    random_state=1729,
    alpha=best_alpha,
    eta0=best_eta0,
    class_weight={0: 0.1, 1: 2}
)

# Train the classifier
sgd_best.fit(X_train, y_train)

# Predict on the test set
y_pred = sgd_best.predict(X_test)

# Get confusion matrix to identify correct predictions for class 1
cm = confusion_matrix(y_test, y_pred)

# Correctly predicted samples of class 1 are in cm[1, 1]
correct_class_1 = cm[1, 1]
print(f"Correctly predicted samples of class 1: {correct_class_1}")

Correctly predicted samples of class 1: 106


In [31]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# Initialize the SVM classifier with given parameters
svm = SVC(
    kernel='rbf',
    decision_function_shape='ovr',
    random_state=1729,
    C=1
)

# Train the model on the training data
svm.fit(X_train, y_train)

# Predict on the test data
y_pred = svm.predict(X_test)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[79 15]
 [ 8 98]]


In [32]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree Classifier with the specified parameters
dtree = DecisionTreeClassifier(
    criterion='entropy',
    splitter='random',
    min_samples_split=4,
    min_impurity_decrease=0.0001,
    random_state=1729
)

# Train the classifier on the training data
dtree.fit(X_train, y_train)

# Get the depth of the tree
tree_depth = dtree.get_depth()
print(f"The resultant depth of the tree is: {tree_depth}")

The resultant depth of the tree is: 12


In [33]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree Classifier with the specified parameters
dtree = DecisionTreeClassifier(
    criterion='entropy',
    splitter='random',
    min_samples_split=4,
    min_impurity_decrease=0.0001,
    random_state=1729
)

# Train the classifier on the training data
dtree.fit(X_train, y_train)

# Get the number of nodes in the tree
num_nodes = dtree.tree_.node_count
print(f"The number of nodes in the tree is: {num_nodes}")


The number of nodes in the tree is: 135


In [34]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree Classifier with the specified parameters
dtree = DecisionTreeClassifier(
    criterion='entropy',
    splitter='random',
    min_samples_split=4,
    min_impurity_decrease=0.0001,
    random_state=1729
)

# Train the classifier on the training data
dtree.fit(X_train, y_train)

# Get the tree structure
tree = dtree.tree_

# The left child of the root is at index 1 (root is index 0)
left_child_index = tree.children_left[0]

# The entropy value at the left child
entropy_left_child = tree.impurity[left_child_index]
print(f"The entropy at the left child of the root is: {entropy_left_child}")


The entropy at the left child of the root is: 0.927617618153467


In [35]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Initialize classifiers
base_estimators = {
    'DecisionTree': DecisionTreeClassifier(random_state=1729),
    'KNeighbors': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(random_state=1729)
}

# Prepare your data (replace X and y with actual dataset)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0., random_state=1729)

# Dictionary to store results
results = {}

# Test each base estimator in BaggingClassifier
for name, estimator in base_estimators.items():
    # Create BaggingClassifier with 20 base estimators
    bagging = BaggingClassifier(estimator=estimator, n_estimators=20, random_state=1729)
    
    # Train the BaggingClassifier
    bagging.fit(X_train, y_train)
    
    # Predict on train and test data
    train_pred = bagging.predict(X_train)
    test_pred = bagging.predict(X_test)
    
    # Calculate accuracy
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    
    # Calculate the absolute difference between train and test accuracy
    abs_diff = abs(train_acc - test_acc)
    
    # Store the results
    results[name] = abs_diff

# Output the base estimator with the lowest absolute difference
best_model = min(results, key=results.get)
print(f"The best base estimator is: {best_model} with an absolute accuracy difference of {results[best_model]}")

The best base estimator is: LogisticRegression with an absolute accuracy difference of 0.02749999999999997


In [36]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Initialize classifiers
classifiers = [
    ('DecisionTree', DecisionTreeClassifier(random_state=1729)),
    ('KNeighbors', KNeighborsClassifier()),
    ('LogisticRegression', LogisticRegression(random_state=1729))
]

# Prepare your data (replace X and y with actual dataset)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1729)

# Create VotingClassifier with the classifiers
voting = VotingClassifier(estimators=classifiers, voting='hard')

# Train the VotingClassifier
voting.fit(X_train, y_train)

# Predict on train and test data
train_pred = voting.predict(X_train)
test_pred = voting.predict(X_test)

# Calculate accuracy
train_acc = accuracy_score(y_train, train_pred)
test_acc = accuracy_score(y_test, test_pred)

# Calculate the absolute difference between train and test accuracy
abs_diff = abs(train_acc - test_acc)

# Print the absolute difference up to 4 decimal places
print(f"The absolute difference in accuracy between train and test is: {abs_diff:.4f}")

The absolute difference in accuracy between train and test is: 0.0587
